### Uncertainty analysis
Using the Monte Carlo Simulations method

In [ ]:
import numpy as np
import rasterio as rio

In [ ]:
# Load the rasters
ksat = rio.open('Rasters/ksat.tif')
wcs = rio.open('Rasters/wcs.tif')
alt_dev = rio.open('Rasters/alt-dev.tif')

In [3]:
# Convert rasters to numpy arrays
ksat_data = ksat.read()
wcs_data = wcs.read()
alt_dev_data = alt_dev.read()

smca_data = {
    'ksat': ksat_data,
    'wcs': wcs_data,
    'alt_dev': alt_dev_data
}


In [4]:
def monte_carlo_simulation(model, data, num_simulations):


SyntaxError: incomplete input (2009632123.py, line 1)

In [ ]:
monte_carlo_simulation(smca_model.smca_model, smca_data, 10)

# Save the results to a file
with open('simulation_results.txt', 'w') as f:
    for result in monte_carlo_simulation(smca_model.smca_model, smca_data, 10):
        f.write(f"{result}\n")

TypeError: 'module' object is not callable. Did you mean: 'smca_model.smca_model(...)'?

In [5]:
weights = {
    'ksat': 0.43,
    'wcs': 0.14,
    'alt_dev': 0.43
}

# Define the type for each criterion
# '1' for benefit, '0' for cost
types = {
    'ksat': '1',
    'wcs': '1',
    'alt_dev': '0'
}

In [10]:
def smca_model(data, weights, types, output_file, profile_file):
    """
    SMCA model implementation.
    
    Parameters:
    ksat (rasterio.DatasetReader): Saturated hydraulic conductivity raster.
    wcs (rasterio.DatasetReader): Water content saturation raster.
    alt_dev (rasterio.DatasetReader): Altitude deviation raster.
    weights (np.ndarray): Weights for each criterion.
    types (np.ndarray): Types of each criterion (1 for benefit, 0 for cost).
    
    Returns:
    np.ndarray: Resulting raster after applying the SMCA model.
    """
    # Check if the data all have the same shape
    if not (data['ksat'].shape == data['wcs'].shape == data['alt_dev'].shape):
        print("ksat shape:", data['ksat'].shape)
        print("wcs shape:", data['wcs'].shape)
        print("alt_dev shape:", data['alt_dev'].shape)
        raise ValueError("All input rasters must have the same shape.")
        
    # Stack the data
    data = np.stack((data['ksat'], data['wcs'], data['alt_dev']), axis=-1)

    # Apply weights and types
    weighted_data = data * weights * types[:, np.newaxis]
    
    # Sum the weighted data
    result = np.sum(weighted_data, axis=-1)
    
    # Save the result to a file
    profile = rio.open(profile_file).profile
    with rio.open(output_file, 'w', **profile) as dst:
        dst.write(result, 1)

    return result

smca_model(
    smca_data,
    weights,
    types,
    output_file='smca_results.tif',
    profile_file='./Rasters/ksat.tif'
)

ksat shape: (1, 342, 339)
wcs shape: (1, 342, 339)
alt_dev shape: (1, 3422, 3398)


ValueError: All input rasters must have the same shape.